<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/time_valuation_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
file_names = os.listdir("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/")
df_names = [x[:-4] for x in file_names]
print(df_names)

dfs = [pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/"+x) for x in file_names]
df_dict = dict(zip(df_names, dfs))
print(df_dict.keys())

['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events']
dict_keys(['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events'])


In [5]:
df_dict['funding_rounds'].head()
# df_dict['organizations'].head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,143236.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,500000.0,USD,NaN,NaN,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,145939.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,California,Menlo Park,series_a,2005-05-01,12700000.0,12700000.0,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,funding_round,facebook-series-b--6fae3958,https://www.crunchbase.com/funding_round/faceb...,273250.0,2007-05-27 06:09:36,2018-02-12 23:30:46,USA,CA,California,Menlo Park,series_b,2006-04-01,27500000.0,27500000.0,USD,502500000.0,502500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-..."
3,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,funding_round,photobucket-series-b--bcd5a63d,https://www.crunchbase.com/funding_round/photo...,149479.0,2007-05-29 11:05:59,2018-02-12 23:27:36,USA,CO,Colorado,Denver,series_b,2006-05-01,10500000.0,10500000.0,USD,NaN,NaN,USD,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN
4,60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,funding_round,geni-series-a--60e6afd9,https://www.crunchbase.com/funding_round/geni-...,313010.0,2007-05-31 20:19:28,2018-02-12 23:41:29,USA,CA,California,West Hollywood,series_a,2007-01-17,NaN,NaN,NaN,10000000.0,10000000.0,USD,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c


In [ ]:
# We should select a set of companies for which there aren't a lot of missing values. But we don't want to overbias either.

# def join_data_per_company(df_dict, company_ids):
#   # Companies 
#   funding_rounds = df_dict['funding_rounds']

#   truncated = funding_rounds[funding_rounds['org_uuid'].isin(company_ids)]
#   return df_dict['funding_rounds'][company_ids]
#   # df_dict['organizations'][company_ids]

company_ids = ['df662812-7f97-0b43-9d3e-12f64f504fbb']
# join_data_per_company(df_dict, company_ids)

In [ ]:
funding_data = deepcopy(
    df_dict["funding_rounds"][['org_uuid', 'announced_on', 'post_money_valuation_usd']]).rename(
    columns={"announced_on": "date", "post_money_valuation_usd": "valuation"})
acquisitions_data = deepcopy(df_dict["acquisitions"][['acquiree_uuid', 'acquired_on', 'price_usd']]).rename(
    columns={"acquiree_uuid": "org_uuid", "acquired_on": "date", "price_usd": "valuation"})
ipos_data = deepcopy(df_dict["ipos"][['org_uuid', 'went_public_on', 'valuation_price_usd']]).rename(
    columns={"went_public_on": "date", "valuation_price_usd": "valuation"})

types = df_dict["acquisitions"][['type']].values
acquisition_types = df_dict["acquisitions"][['acquisition_type']].values
acquisitions_data['event'] = [types[x][0] if not isinstance(acquisition_types[x][0], str) 
                              else acquisition_types[x][0] for x in range(len(acquisitions_data))]
funding_data['event'] = pd.Series([x[0] for x in df_dict["funding_rounds"]['name'].str.split(" - ")])
ipos_data['event'] = df_dict['ipos']['stock_exchange_symbol']

timelines = pd.concat([funding_data, acquisitions_data, ipos_data])

# Normalize dates

timelines.sort_values(by=['org_uuid', 'date'], inplace=True)
timelines.reset_index(inplace=True, drop=True)

timelines.set_index(['org_uuid'])
grouped_timelines = timelines.groupby(['org_uuid'])

# Normalization of dependent variables

### Time Normalization

In [ ]:
import datetime
company_timelines = {}

milli_per_day = 86400  # Used for converting Unix time

for company_id in company_ids:
  timeline = grouped_timelines.get_group(company_id).copy()
  timeline = timeline[timeline['valuation'].notnull()]               # We only care about times at which there is a valuation of the company.
  timeline.reset_index(inplace=True, drop=True)

  day_zero = datetime.datetime.strptime(timeline['date'].iloc[0], '%Y-%m-%d').timestamp()
  parsed_dates = pd.Series([datetime.datetime.strptime(x, '%Y-%m-%d').timestamp() for x in timeline['date']])
  norm_dates = (parsed_dates - day_zero)/milli_per_day
  timeline['norm_dates'] = norm_dates
  company_timelines[company_id] = timeline
  # print(timeline)
  # print(norm_dates)

### Value Normalization

In [ ]:
for company_id, timeline in company_timelines.items():
  initial_valuation = timeline['valuation'].iloc[0]
  norm_valuations = pd.Series([np.log(value/initial_valuation) for value in timeline['valuation']])
  timeline['norm_valuations'] = norm_valuations

# Selecting Independent Vars

In [8]:
df_dict['organizations'].head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,domain,homepage_url,country_code,state_code,region,city,address,postal_code,status,short_description,category_list,category_groups_list,num_funding_rounds,total_funding_usd,total_funding,total_funding_currency_code,founded_on,last_funding_on,closed_on,employee_count,email,phone,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,123607.0,2007-05-25 13:51:27,2019-06-24 22:19:25,NaN,company,wetpaint.com,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,acquired,Wetpaint offers an online social publishing pl...,"Publishing,Social Media,Social Media Management","Content and Publishing,Internet Services,Media...",3.0,3.975000e+07,3.975000e+07,USD,2005-06-01,2008-05-19,NaN,51-100,info@wetpaint.com,206-859-6300,https://www.facebook.com/Wetpaint,https://www.linkedin.com/company/wetpaint,https://twitter.com/wetpainttv,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,8777.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",zoho.com,https://www.zoho.com/,USA,CA,California,Pleasanton,4141 Hacienda Drive,94588,operating,"Zoho offers a suite of business, collaboration...","Cloud Computing,Collaboration,CRM,Developer To...","Information Technology,Internet Services,Priva...",NaN,NaN,NaN,NaN,1996-09-15,NaN,NaN,1001-5000,info@zohocorp.com,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,1.0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,11537.0,2007-05-26 03:03:23,2018-12-10 10:09:14,"Digg Holdings, LLC",company,digg.com,http://www.digg.com,USA,NY,New York,New York,NaN,NaN,acquired,Digg Inc. operates a website that enables its ...,"Internet,Social Media,Social Network","Internet Services,Media and Entertainment",6.0,4.900000e+07,4.900000e+07,USD,2004-10-11,2016-09-13,NaN,51-100,feedback@digg.com,877-342-7222,http://www.facebook.com/digg,http://www.linkedin.com/company/digg,http://twitter.com/digg,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,6844.0,2007-05-26 03:21:34,2019-06-19 12:17:48,NaN,investor,omidyar.com,http://www.omidyar.com,USA,CA,California,Redwood City,1991 Broadway Suite 200,94063,operating,Omidyar Network is an investment firm.,"Enterprise Software,Financial Services,Venture...","Financial Services,Lending and Investments,Sof...",NaN,NaN,NaN,NaN,2004-01-01,NaN,NaN,101-250,info@omidyar.com,650.482.2500,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,investor,33.0
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,15.0,2007-05-26 04:22:15,2020-03-30 18:33:35,"Facebook, Inc.","investor,company",facebook.com,http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,ipo,Facebook is an online social networking servic...,"E-Commerce,Mobile Apps,Social,Social Media,Soc...","Apps,Commerce and Shopping,Content and Publish...",15.0,2.335700e+09,2.335700e+09,USD,2004-02-04,2013-06-30,NaN,10000+,NaN,NaN,https://www.facebook.com/facebook/,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN


## Build a Vocabulary

In [21]:
import scipy.sparse
import collections

org_info = df_dict['organizations']

# Filter down the data purely to investees
org_info = org_info[org_info['primary_role'] == 'company']

# Select organization data
# Clean the df

comma_sep_list_vars = ['category_list', 'category_groups_list']
categorical_vars = ['region']
other_vars = ['uuid']
org_info_selected = org_info[other_vars + categorical_vars + comma_sep_list_vars]
org_info_selected = org_info_selected.dropna()

# Map of word
vocabulary = collections.Counter()
cooccurrence_counts = collections.Counter()

## TODO: Add more columns to the company data, like joining founder + degree earned

iter = 0
for (i, row) in org_info_selected.iterrows():
  iter += 1
  if iter > 100:
    break
  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      tokens = []
      for token_list in token_lists:
        for token in token_list:
          tokens.append(token)

      for token in tokens:
        vocabulary[token] += 1
        for other_token in tokens:
          cooccurrence_counts[(token, other_token)] += 1

# Filter low count words
low_count_threshold = 5
keys_to_remove = []
for word, count in vocabulary.items():
  if count < low_count_threshold:
    keys_to_remove.append(word)

for key in keys_to_remove:
  del vocabulary[key]

# Now rank and filter by document frequency. The top half frequent words will be dropped
key_list = list(vocabulary.keys())
chop = sorted(key_list, key=lambda x: vocabulary[x])[int(len(key_list)/2):]
for key in chop:
  del vocabulary[key]

# Create index to word mapping
word_to_index = {}
word_index = 0
for word, count in vocabulary.items():
  word_to_index[word] = word_index
  word_index += 1


print(word_to_index)
ica_precursor = np.zeros((word_index, word_index))
for word in vocabulary.keys():
  for other_word in vocabulary.keys():
    ica_precursor[word_to_index[word], word_to_index[other_word]] = cooccurrence_counts[(word, other_word)]

print(ica_precursor)


# from google.colab import files
# files.download('data.dat')

{'Management': 0, 'Enterprise': 1, 'Network': 2, 'Security': 3, 'E-Commerce': 4, 'Streaming': 5, 'Commerce': 6, 'Shopping': 7, 'Web': 8, 'Hosting': 9, 'Community': 10, 'Lifestyle': 11, 'Platforms': 12, 'Search': 13, 'Engine': 14, 'TV': 15, 'Design': 16, 'Financial': 17, 'Broadcasting': 18, 'Gaming': 19, 'Finance': 20, 'VoIP': 21, 'Travel': 22, 'Tourism': 23}
[[ 7.  1.  1.  1.  0.  0.  0.  0.  2.  1.  0.  0.  0.  1.  1.  1.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 1.  5.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 1.  1.  8.  1.  1.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  1.  0.]
 [ 1.  1.  1. 11.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  6.  1.  0.  1.  1.  1.  0.  0.  0.  2.  2.  0.  0.  0.
   0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.
   1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  7.  7.

## Add other vars

In [ ]:

# data_dense_rows = []
# data_sparse_rows = []
# dummy_list_var_index = 0
# list_var_set = set()

for i, row in org_info_selected.iterrows():
  sparse_out = {}
  dense_out = {}
  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      value_types = []
      for token_list in token_lists:
        for token in token_list:
          value_types.append(token)
      for value_type in value_types:
        sparse_out[value_type] = 1
        list_var_set.add(value_type)
    elif col in categorical_vars:
      value_type = row[col]
      sparse_out[value_type] = 1
    else:
      dense_out[col] = row[col]
  dense_out['uuid'] = row['uuid']
  data_sparse_rows.append(pd.DataFrame.from_dict(sparse_out, orient='index').astype(pd.SparseDtype(int, np.nan)))
  data_dense_rows.append(dense_out)
  print(len(list_var_set))

print("Concatenating")
sparse_output = pd.concat(data_sparse_rows, axis=0, ignore_index=True)
dense_output = pd.concat(data_dense_rows, axis=0, ignore_index=True)
output.head()

# Modelling the problem in the scope of a strategy

Suppose my strategy was mostly based on my expectations of what value a startup reached by day N. 

I could define N to be [0, 500, 1000, 1500], and that means I am making decisions based on predicted valuation at days 0, 500, 1000, 1500. 

For each N, we can train a regressor that predicts valuation at day N. If day N
does not have a specific price, we interpolate linearly between price points.

In [ ]:
prediction_dates = [500]

